# Recover Boundary velocity (Least Square) 
> jeonghyeop kim (2021/7/28) <font color=red>(v. testing)</font>
> \
> My first inversion code written in python

This code uses:
1. data vector: **`vel_on_boundary_wrt_URcorner.gmt`** & \
2. G-matrix components: **`vel_BC_%d_%d.gmt`**, (% 001..024; 100, 010, 001) \
          100 : x-rotation; 010 : y-rotation; 001 : z-rotation
3. For **`vel_BC_%d_%d.gmt`**, I don't know how Ali generates them yet. Talk to him or Bill
4. assumed that UCERF3 velocity is errorless


In [2]:
#import numpy and pandas
import numpy as np
import pandas as pd
import scipy 
pd.__version__
#np.__version__

'1.0.5'

### `STEP1`: **BUILD a data vector,  $\vec{d}$**

In [3]:
# load input file
inputFILE = "vel_on_boundary_wrt_URcorner.gmt" # input file name
df_input = pd.read_csv(inputFILE, header = None, sep =' ')
df_input.columns = ['lon','lat','ve','vn','se','sn','corr']

In [4]:
# BUILD a 'data' vector d along with coordinate information

df_data_x = df_input.iloc[:,[0,1,2]]  # saved vx data on the boudnary
df_data_y = df_input.iloc[:,[0,1,3]]  # saved vn data on the boundary

df_data_x=df_data_x.rename(columns ={'ve': 'velo'}) #column name change
df_data_y=df_data_y.rename(columns ={'vn': 'velo'}) #column name change

df_data_x['flag']=np.array([1] * len(df_data_x)) # 1 = velocity east-component
df_data_y['flag']=np.array([2] * len(df_data_y)) # 2 = velocity north-component

# !! SORT_VALUES !! # lat ascending first, and then lon ascending.
# This step is very important to build the G matrix, G, which
# has rows correspoding to the rows of the data vector, d, that have
# the same coordinates!

df_data_x=df_data_x.sort_values(['lat', 'lon'], ascending=[True, True])
df_data_y=df_data_y.sort_values(['lat', 'lon'], ascending=[True, True])


# MERGE two columns (n*1) into a new column (2n*1)
# > ignore_index = True : 
# >   have one continuous index numbers,
# >     ignorning each of the two dfs original indices

frames=[df_data_x,df_data_y]
df_data=pd.concat(frames,ignore_index=True) # merge the two dataFrames into one


df_data=df_data['velo']

#df_data #to check the data vector

NOTE:   
>If your data have errors, \
build a **W** matrix in which its diagonal elements \
are of a vector made of 1/errors (in order).  \
Your new data vector **d<sub>w</sub>** = **W** **d**

### `STEP2`: **BUILD G-Matrix, $\bar{\bar{G}}$**

> Comment on the read_csv(sep option) below: \
> **`'(?:,|\s+)'`** - is a RegEx for selecting either comma or any number of consecutive spaces/tabs \
> See the answer at StackOverFlow [click here](https://stackoverflow.com/questions/43784422/pandas-error-when-reading-csv-file-using-sep-and-comment-arguments)

In [7]:
df_G = pd.DataFrame(index = range(len(df_data))) 
# Make a blank G matrix


print('How many files do you have for the boudnary basis functions? :')
HowMany = input()
HowMany = int(HowMany)
#24 for the boundary condition (7/28/2021)

for i in range(1,HowMany+1): # i in range(0+1,how many files+1?); i in {001 002 003 ... 999} 

    inputfile_xrot = "vel_BC_"+str(f"{i:03}")+"_100.gmt" # x-rot : 100
    inputfile_yrot = "vel_BC_"+str(f"{i:03}")+"_010.gmt" # y-rot : 010
    inputfile_zrot = "vel_BC_"+str(f"{i:03}")+"_001.gmt" # z-rot : 001

# READ files in order {xrot1, yrot1, zrot1, ..., xrotHowMany, yrotHowMany, zrotHowMany}

    df_xrot=pd.read_csv(inputfile_xrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_yrot=pd.read_csv(inputfile_yrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')
    df_zrot=pd.read_csv(inputfile_zrot ,header=None, sep=r'(?:,|\s+)', 
                           comment='#', engine='python')

# CHANGE the column names 

    df_xrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_yrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    df_zrot.columns = ['lon','lat','ve','vn','se','sn','corr']
    
# BUILD a column vector Gx (i)

    df_xrot_x = df_xrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_xrot_y = df_xrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_xrot_x=df_xrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_xrot_y=df_xrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_xrot_x['flag']=np.array([1] * len(df_xrot_x)) # 1 = velocity east-component
    df_xrot_y['flag']=np.array([2] * len(df_xrot_y)) # 2 = velocity north-component
    
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_xrot_x=df_xrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_xrot_y=df_xrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices   
    frames_Gx=[df_xrot_x,df_xrot_y]
    df_Gx=pd.concat(frames_Gx,ignore_index=True) # merge the two dataFrames into one

    
    
# BUILD a column vector Gy (i)

    df_yrot_x = df_yrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_yrot_y = df_yrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_yrot_x=df_yrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_yrot_y=df_yrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_yrot_x['flag']=np.array([1] * len(df_yrot_x)) # 1 = velocity east-component
    df_yrot_y['flag']=np.array([2] * len(df_yrot_y)) # 2 = velocity north-component

    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_yrot_x=df_yrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_yrot_y=df_yrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gy=[df_yrot_x,df_yrot_y]
    df_Gy=pd.concat(frames_Gy,ignore_index=True) # merge the two dataFrames into one
    
    
# BUILD a column vector Gz (i)

    df_zrot_x = df_zrot.iloc[:,[0,1,2]]  # saved vx basis function on the boudnary
    df_zrot_y = df_zrot.iloc[:,[0,1,3]]  # saved vn basis function on the boundary

    df_zrot_x=df_zrot_x.rename(columns ={'ve': 'velo'}) #column name change
    df_zrot_y=df_zrot_y.rename(columns ={'vn': 'velo'}) #column name change

    df_zrot_x['flag']=np.array([1] * len(df_zrot_x)) # 1 = velocity east-component
    df_zrot_y['flag']=np.array([2] * len(df_zrot_y)) # 2 = velocity north-component
   
    # !! SORT_VALUES !! # lat ascending first, and then lon ascending.
    # This step is very important to build the G matrix, G, which
    # has rows correspoding to the rows of the data vector, d, that have
    # the same coordinates!
    df_zrot_x=df_zrot_x.sort_values(['lat', 'lon'], ascending=[True, True])
    df_zrot_y=df_zrot_y.sort_values(['lat', 'lon'], ascending=[True, True])
    
    # MERGE two columns (n*1) into a new column (2n*1)
    # > ignore_index = True : 
    # >   have one continuous index numbers,
    # >     ignorning each of the two dfs original indices
    frames_Gz=[df_zrot_x,df_zrot_y]
    df_Gz=pd.concat(frames_Gz,ignore_index=True) # merge the two dataFrames into one
    
    
# SAVE G-matrix
# Gmatrix = [Gxrot(1) Gyrot(1) Gzrot(1) ... Gxrot(HowMany) Gyrot(HowMany) Gzrot(HowMany)]
    
    df_G["G_x"+str(i)] = df_Gx.loc[:,['velo']]
    df_G["G_y"+str(i)] = df_Gy.loc[:,['velo']]
    df_G["G_z"+str(i)] = df_Gz.loc[:,['velo']]

How many files do you have for the boudnary basis functions? :


 24


In [8]:
#Check the G-matrix
# outputFILE='Gmatrix_test.dat'
# df_G.to_csv(outputFILE, header=None, index=None, sep=' ',float_format='%f6.3')
# df_G


### `STEP3`: Inversion (Least Square Sense) 
>  **d** = **G** **m** \
>  **G'** **d** = **G'** **G** **m** \
>  (**G'** **G**)<sup>-1</sup> **G'** **d** = $\hat{m}$

 One can try <font color=red>weighting</font> and <font color=red>damping</font> later! \
 For now (7/28/2021), this code just applies a simplest LSM. 
 
**Pandas: For element-wise operation syntax,** [click this](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.multiply.html)

In [9]:
# G'
df_G_prime = df_G.transpose() 

# G'G
# >Two different ways to compute a matrix multiplication
# >1st method
GpG1=df_G_prime.dot(df_G) #G'G
# >2nd method
GpG2=df_G_prime @ df_G #G'G
# >These results are same.
# >Let's take the second one as G'G
GpG = GpG2 #GpG is G'G


# inv(G'G)
# > Two different ways to obtain inverse matrix
# > 1st method: np.linalg.inv 
df_inv_GpG = pd.DataFrame(np.linalg.inv(GpG.to_numpy()), GpG.columns, GpG.index)
# > 2nd method: np.linalg.pinv (Moore-Penrose inverse (SVD))
df_pinv_GpG = pd.DataFrame(np.linalg.pinv(GpG.to_numpy()), GpG.columns, GpG.index)


# inv(G'G)*G'*d = model(LSM)
df_model1=df_inv_GpG@df_G_prime@df_data
df_model2=df_pinv_GpG@df_G_prime@df_data

In [10]:
#The difference is very small 
df_model1-df_model2

G_x1    -0.001435
G_y1    -0.002874
G_z1     0.002163
G_x2    -0.001825
G_y2    -0.003654
           ...   
G_y23    0.000003
G_z23   -0.000002
G_x24    0.000052
G_y24    0.000103
G_z24   -0.000078
Length: 72, dtype: float64

In [11]:
df_data_predicted = df_G@df_model1 #data predicted.

In [12]:
#norm2 misfit
df_norm2=(df_data-df_data_predicted)**2
df_norm2=df_norm2.to_numpy().sum()
df_norm2=np.lib.scimath.sqrt(df_norm2)
df_norm2

0.03189875729010651

In [13]:
# SAVE predicted velocity values on the boundary in *.gmt file.

num_velo_point=len(df_data_predicted)/2
num_velo_point=int(num_velo_point)
df_prediction_vx=df_data_predicted.iloc[0:num_velo_point] #vx predicted
df_prediction_vy=df_data_predicted.iloc[num_velo_point:2*num_velo_point] #vy predicted
df_prediction_vy=df_prediction_vy.reset_index(drop=True) #index starts from 0, and drop old index numbers


# Use 'df_data_x' (**already sorted by lat and lon**) as a frame to save the predicted values
# 'df_data_x' has columns : lon lat vel flag
df_save = df_data_x 
df_save['ve'] = df_prediction_vx #append a new column named ve
df_save['vn'] = df_prediction_vy #append a new column named vn
df_save['se'] = np.array([0] * len(df_save)) #append a new column named se : zero vector
df_save['sn'] = np.array([0] * len(df_save)) #append a new column named sn : zero vector
df_save['corr'] = np.array([0] * len(df_save)) #append a new column named corr : zero vector
df_save=df_save.loc[:,['lon','lat','ve','vn','se','sn','corr']] #slice the seven columns of interest!


#Save output : "vel_on_boundary_wrt_URcorner_pred.gmt"
outputFILE="vel_on_boundary_wrt_URcorner_pred.gmt"
df_save.to_csv(outputFILE, header=None, index=None, sep=' ',float_format='%g')

In [18]:
%%bash
#Plot!
#psvelo (GMT4 syntax) -Alinethickness/?/arrowthickness
sh image_boundary_data_wrt_URC_obs_pred
gmt psconvert boundary_data_wrt_URC_pred.ps -Tf -A
open boundary_data_wrt_URC_pred.pdf

psbasemap [INFORMATION]: Constructing the basemap
psbasemap [INFORMATION]: Central meridian not given, default to -116.61
psbasemap [INFORMATION]: Map scale is 1.23688 km per cm or 1:123688.
pscoast [INFORMATION]: Central meridian not given, default to -116.61
pscoast [INFORMATION]: Map scale is 1.23688 km per cm or 1:123688.
pscoast [INFORMATION]: Selected ice front line as Antarctica coastline
pscoast [INFORMATION]: GSHHG version 2.3.7
pscoast [INFORMATION]: Derived from World Vector Shoreline, CIA WDB-II, and Atlas of the Cryosphere
pscoast [INFORMATION]: Processed by Paul Wessel and Walter H. F. Smith, 1994-2017
pscoast [INFORMATION]: Adding Borders...psxy [INFORMATION]: Processing input table data
psxy [INFORMATION]: Central meridian not given, default to -116.61
psxy [INFORMATION]: Map scale is 1.23688 km per cm or 1:123688.
psxy [INFORMATION]: Reading Data Table from Stream 7fff889fbe50
psxy [INFORMATION]: Plotting segment 0
pstext [INFORMATION]: Processing input text table data